# メインコード
サーバサイドはこれを実行すればOK．

In [1]:
from bottle import get, post, run, HTTPResponse
import requests
import csv
import json
import re
import urllib.parse
import pymysql

In [2]:
class UseMySQL:
    def __init__(self,
                 host='localhost', user='root', password='root',
                 db='ng_1706', charset='utf8'):
        self.connection = pymysql.connect(
            host=host, user=user, passwd=password, db=db, charset=charset, cursorclass=pymysql.cursors.DictCursor)
        cursor = self.connection.cursor()
        sql = "show tables like 'find_phrase'"
        numrows = cursor.execute(sql)
        if numrows < 1:
            sql = 'create table find_phrase(before_phrase varchar(100) not null, after_phrase varchar(100) not null, pattern varchar(20) not null, create_at datetime default current_timestamp on update current_timestamp, primary key (before_phrase, after_phrase, pattern))'
            cursor.execute(sql)
            self.connection.commit()
        
    def get_from_find_phrase_by_date(self, date):
        cursor = self.connection.cursor()
        sql = 'select before_phrase, after_phrase from find_phrase where create_at = ' + date
        print(sql)
        cursor.execute(sql)
        return cursor.fetchall()
        
    def insert_to_find_phrase(self, before, after, pattern):
        cursor = self.connection.cursor()
        sql = "select * from find_phrase where before_phrase = '" + before + "' and after_phrase = '" + after + "' and pattern = '" + pattern + "'"
        numrows = cursor.execute(sql)
        if numrows < 1:
            sql = 'insert into find_phrase(before_phrase, after_phrase, pattern) value ("' + before + '", "' + after + '", "' + pattern + '")'
            cursor.execute(sql)
            self.connection.commit()
        
usemysql = UseMySQL()

## 便利な関数群
各API部から共通で呼び出される機能をここにまとめています

In [3]:
def csvreader(fileaddr = '../data/jk.csv'):
    csvlist = []
    with open(fileaddr, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            csvlist.append((row[0],row[1]))
    
    return csvlist

In [4]:
def strextractor(string, pattern):
    r = ''
    matchOB = re.search(pattern , string)
    if matchOB:
        r = matchOB.group()
    print(matchOB)
    return r

In [5]:
def requestjsonstr2dic(data):
    jsonstr = ''
    for k,v in data.items():
        jsonstr = k
    dictionary = json.loads(jsonstr)
    return dictionary

In [6]:
def exchange_common(message, exchange = 'jk', reverse = False):
    find_phrase = {}
    exchange_tapple = csvreader('../data/' + exchange + '.csv')
    for x, y in exchange_tapple:
        if reverse:
            after = x
            before = y
        else:
            before = x
            after = y
        check_fp = message.find(before)
        if check_fp > -1 :
            find_phrase[before] = after
        message = message.replace(before, after)
        
    for before, after in find_phrase.items():
        if reverse:
            usemysql.insert_to_find_phrase(after, before, exchange)
        else:
            usemysql.insert_to_find_phrase(before, after, exchange)
        
    return message, find_phrase

### phrasefromdate
変換履歴を日時から取得する

In [7]:
@post('/phrasefromdate/<year>/<month>/<day>')
def phrasefromdate(year, month, day):
    date = year + '-' + month + '-' + day
    find_phrase = usemysql.get_from_find_phrase_by_date(date)
    
    
    body = {"status": "200", "find_phrase": find_phrase}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

## Exchange
文字列を変換して返すAPI群

### exhangejk
jk用語を共通語に変換する

getメソッド
パラメータ
message : 変換元のメッセージ

返り値
appliction/json形式
status  : ステータスコード
message : 変換したメッセージ

In [8]:
@post('/exchangejk/<message>')
def p_exchangejk(message):
    message, find_phrase = exchange_common(message, exchange = 'jk', reverse = False)
    
    body = {"status": "200", "message": message, "find_phrase": find_phrase}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

In [9]:
@post('/reverse_exchangejk/<message>')
def p_reverse_exchangejk(message):
    message, find_phrase = exchange_common(message, exchange = 'jk', reverse = True)
    
    body = {"status": "200", "message": message, "find_phrase": find_phrase}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

### exhangeit
it技術者が使う用語を共通語に変換する

getメソッド
パラメータ
message : 変換元のメッセージ

返り値
appliction/json形式
status  : ステータスコード
message : 変換したメッセージ

In [10]:
@post('/exchangeit/<message>')
def p_exchangeit(message):
    message, find_phrase = exchange_common(message, exchange = 'it', reverse = False)
    
    body = {"status": "200", "message": message, "find_phrase": find_phrase}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

In [11]:
@post('/reverse_exchangeit/<message>')
def p_exchangeit(message):
    message, find_phrase = exchange_common(message, exchange = 'it', reverse = True)
    
    body = {"status": "200", "message": message, "find_phrase": find_phrase}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

## 実行
以上のAPIをポート8080番で実行する

In [ ]:
run(host='0.0.0.0', port=8080, debug=True)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.

106.129.138.219 - - [22/Oct/2017 04:28:58] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20 HTTP/1.1" 200 77
106.129.138.219 - - [22/Oct/2017 04:28:58] "POST /exchangejk/%20%E5%BD%BC%20%E6%9B%B0%E3%81%8F%20 HTTP/1.1" 200 72
106.129.138.219 - - [22/Oct/2017 04:28:58] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20 HTTP/1.1" 200 97
106.129.138.219 - - [22/Oct/2017 04:28:58] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%E8%A1%86%20 HTTP/1.1" 200 96
106.129.138.219 - - [22/Oct/2017 04:28:59] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20 HTTP/1.1" 200 97
106.129.138.219 - - [22/Oct/2017 04:28:59] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20 HTTP/1.1" 200 97
106.129.138.219 - - [22/Oct/2017 0

106.129.138.219 - - [22/Oct/2017 04:29:18] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%E3%81%8C%20%20%E8%B2%A0%E3%81%91%E3%81%9A%E5%AB%8C%E3%81%84%20%E3%81%AA%20%E7%94%9F%E3%83%8E%E6%9D%BE%E5%8E%9F%20%E3%81%AA%E3%81%A9%20 HTTP/1.1" 200 386
106.129.138.219 - - [22/Oct/2017 04:29:18] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%E3%81%8C%20%20%E3%83%90%E3%83%83%E3%83%86%E3%83%AA%E3%83%BC%20%E9%81%B8%E3%81%B0%E3%82%8C%20%E3%81%AA%E3%81%84%20%E3%81%91%E3%81%A9%20%E3%82%8F%E3%81%96%E3%82%8F%E3%81%96%20%E6%97%A5%E5%92%8C%20 HTTP/1.1" 200 424
106.129.138.219 - - [22/Oct/2017 04:29:18] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%2

106.129.138.219 - - [22/Oct/2017 04:29:27] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%E3%81%8C%20%20%E8%B2%A0%E3%81%91%E3%81%9A%E5%AB%8C%E3%81%84%20%E3%81%AA%20%E7%94%9F%E3%81%AE%E6%9D%BE%E5%8E%9F%20%EF%BC%A1%EF%BD%84%EF%BD%8F%EF%BD%82%EF%BD%85%20%E3%82%88%E3%82%8A%E3%82%82%20%20%E3%81%AF%E3%81%84%20%20%E6%9C%9D%20%20%E5%BA%83%E3%81%8F%20%E3%81%99%E3%82%8B%E3%81%A3%E3%81%A6%20%E8%A8%80%E3%81%A3%E3%81%A6%E3%81%9F%20%E3%81%91%E3%81%A9%20 HTTP/1.1" 200 522
106.129.138.219 - - [22/Oct/2017 04:29:28] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%E3%81%8C%20%20%E8%B2%A0%E3%81%91%E3%81%9A%E5%AB%8C%E3%81%84%20%E3%81%AA%20%E7%94%9F%E3%81%AE%E6%9D%BE%E5%8E%9F%20%EF%BC%A1%EF%BD%84%EF%BD%8F%EF%BD%82%EF%BD%85%20%E3%82%

106.129.138.219 - - [22/Oct/2017 04:29:32] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%E3%81%8C%20%20%E8%B2%A0%E3%81%91%E3%81%9A%E5%AB%8C%E3%81%84%20%E3%81%AA%20%E7%94%9F%E3%81%AE%E6%9D%BE%E5%8E%9F%20%EF%BC%A1%EF%BD%84%EF%BD%8F%EF%BD%82%EF%BD%85%20%E3%82%88%E3%82%8A%E3%82%82%20%20%E3%81%AF%E3%81%84%20%20%E6%9C%9D%20%20%E5%BA%83%E3%81%8F%20%E3%81%99%E3%82%8B%E3%81%A3%E3%81%A6%20%E8%A8%80%E3%81%A3%E3%81%A6%E3%81%9F%20%E3%81%91%E3%81%A9%20%E3%81%8A%E6%AF%8D%E3%81%95%E3%82%93%20%20%E5%8D%B1%E3%81%AA%E3%81%84%20%E3%81%A9%E3%82%8C%E3%81%8F%E3%82%89%E3%81%84%20%20%E5%BF%83%20%E3%81%8C%20%E3%81%82%E3%82%8B%20 HTTP/1.1" 200 626
106.129.138.219 - - [22/Oct/2017 04:29:32] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%

106.129.138.219 - - [22/Oct/2017 04:29:41] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%A4%BA%20%E3%81%95%E3%82%8C%E3%81%AA%E3%81%84%20%20%E8%89%B2%20%E3%81%8C%20%20%E8%B2%A0%E3%81%91%E3%81%9A%E5%AB%8C%E3%81%84%20%E3%81%AA%20%E7%94%9F%E3%81%AE%E6%9D%BE%E5%8E%9F%20%EF%BC%A1%EF%BD%84%EF%BD%8F%EF%BD%82%EF%BD%85%20%E3%82%88%E3%82%8A%E3%82%82%20%20%E3%81%AF%E3%81%84%20%20%E6%9C%9D%20%20%E5%BA%83%E3%81%8F%20%E3%81%99%E3%82%8B%E3%81%A3%E3%81%A6%20%E8%A8%80%E3%81%A3%E3%81%A6%E3%81%9F%20%E3%81%91%E3%81%A9%20%E3%81%8A%E6%AF%8D%E3%81%95%E3%82%93%20%20%E5%8D%B1%E3%81%AA%E3%81%84%20%E3%81%A9%E3%82%8C%E3%81%8F%E3%82%89%E3%81%84%20%20%E5%BF%83%20%E3%81%8C%20%E3%81%82%E3%82%8B%20%20%E6%8C%AF%E3%82%8A%E8%BE%BC%E3%81%BF%20%E3%81%97%E3%81%9F%20 HTTP/1.1" 200 665
106.129.138.219 - - [22/Oct/2017 04:29:41] "POST /exchangejk/%20%E3%81%8B%E3%82%8F%E3%81%84%E3%81%84%20%E8%8B%A5%E3%81%84%20%E5%AD%90%20%20%E5%8D%8D%20%20%E8%A1%A8%E7%

...

106.129.121.5 - - [22/Oct/2017 05:07:22] "POST /exchangejk/%20%E3%82%B0%E3%83%AD%E3%83%BC%E3%83%90%E3%83%AB%E5%8C%96%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%20%E4%BC%9A%E7%A4%BE%20%E3%81%A7%20%E8%B2%B7%E3%81%88%E3%82%8B%20%E6%89%8B%E7%B6%9A%E3%81%8D%20%E3%81%A7%20%E5%AE%9F%E9%9A%9B%20%E3%81%AB%20%E3%81%A9%E3%82%93%E3%81%AA%20%E3%82%82%E3%81%AE%20%E3%81%8C%20%E3%81%A7%E3%81%8D%E3%81%9F%20%E3%81%8B%20%E3%81%A3%E3%81%A6%20%E3%81%84%E3%81%86%20%E3%81%AE%20%E3%82%92%20%E8%A6%8B%E3%81%9B%E3%81%9F%E3%81%84%20%20%E5%8B%95%E7%94%BB%20%E3%82%92%20%E3%81%A1%E3%82%87%E3%81%A3%E3%81%A8%20%E8%A6%8B%E3%81%A6%20%E3%81%84%E3%81%BE%E3%81%97%E3%81%9F%20%20%E9%95%B7%E3%81%84%20%20%E3%81%9D%E3%81%86%20%20%E3%81%A9%E3%81%86%20%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%20%E3%81%8B%20%E3%80%82%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%AE%20%E3%81%86%E3%81%A1%20%E3%81%95%E3%81%9B%E3%82%8B%20%E3%81%A7%E3%82%82%20%E3%81%99%E3%82%8B%20%E3%81%8B%E3%81%AA%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%8C%20%E3%81%98%E3%82%83%E3%81%A

106.129.121.5 - - [22/Oct/2017 05:07:23] "POST /exchangejk/%20%E3%82%B0%E3%83%AD%E3%83%BC%E3%83%90%E3%83%AB%E5%8C%96%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%20%E4%BC%9A%E7%A4%BE%20%E3%81%A7%20%E8%B2%B7%E3%81%88%E3%82%8B%20%E6%89%8B%E7%B6%9A%E3%81%8D%20%E3%81%A7%20%E5%AE%9F%E9%9A%9B%20%E3%81%AB%20%E3%81%A9%E3%82%93%E3%81%AA%20%E3%82%82%E3%81%AE%20%E3%81%8C%20%E3%81%A7%E3%81%8D%E3%81%9F%20%E3%81%8B%20%E3%81%A3%E3%81%A6%20%E3%81%84%E3%81%86%20%E3%81%AE%20%E3%82%92%20%E8%A6%8B%E3%81%9B%E3%81%9F%E3%81%84%20%20%E5%8B%95%E7%94%BB%20%E3%82%92%20%E3%81%A1%E3%82%87%E3%81%A3%E3%81%A8%20%E8%A6%8B%E3%81%A6%20%E3%81%84%E3%81%BE%E3%81%97%E3%81%9F%20%20%E9%95%B7%E3%81%84%20%20%E3%81%9D%E3%81%86%20%20%E3%81%A9%E3%81%86%20%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%20%E3%81%8B%20%E3%80%82%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%AE%20%E3%81%86%E3%81%A1%20%E3%81%95%E3%81%9B%E3%82%8B%20%E3%81%A7%E3%82%82%20%E3%81%99%E3%82%8B%20%E3%81%8B%E3%81%AA%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%8C%20%E3%81%98%E3%82%83%E3%81%A

106.129.121.5 - - [22/Oct/2017 05:07:25] "POST /exchangejk/%20%E3%82%B0%E3%83%AD%E3%83%BC%E3%83%90%E3%83%AB%E5%8C%96%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%20%E4%BC%9A%E7%A4%BE%20%E3%81%A7%20%E8%B2%B7%E3%81%88%E3%82%8B%20%E6%89%8B%E7%B6%9A%E3%81%8D%20%E3%81%A7%20%E5%AE%9F%E9%9A%9B%20%E3%81%AB%20%E3%81%A9%E3%82%93%E3%81%AA%20%E3%82%82%E3%81%AE%20%E3%81%8C%20%E3%81%A7%E3%81%8D%E3%81%9F%20%E3%81%8B%20%E3%81%A3%E3%81%A6%20%E3%81%84%E3%81%86%20%E3%81%AE%20%E3%82%92%20%E8%A6%8B%E3%81%9B%E3%81%9F%E3%81%84%20%20%E5%8B%95%E7%94%BB%20%E3%82%92%20%E3%81%A1%E3%82%87%E3%81%A3%E3%81%A8%20%E8%A6%8B%E3%81%A6%20%E3%81%84%E3%81%BE%E3%81%97%E3%81%9F%20%20%E9%95%B7%E3%81%84%20%20%E3%81%9D%E3%81%86%20%20%E3%81%A9%E3%81%86%20%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%20%E3%81%8B%20%E3%80%82%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%AE%20%E3%81%86%E3%81%A1%20%E3%81%95%E3%81%9B%E3%82%8B%20%E3%81%A7%E3%82%82%20%E3%81%99%E3%82%8B%20%E3%81%8B%E3%81%AA%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%8C%20%E3%81%98%E3%82%83%E3%81%A

106.129.121.5 - - [22/Oct/2017 05:07:26] "POST /exchangejk/%20%E3%82%B0%E3%83%AD%E3%83%BC%E3%83%90%E3%83%AB%E5%8C%96%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%20%E4%BC%9A%E7%A4%BE%20%E3%81%A7%20%E8%B2%B7%E3%81%88%E3%82%8B%20%E6%89%8B%E7%B6%9A%E3%81%8D%20%E3%81%A7%20%E5%AE%9F%E9%9A%9B%20%E3%81%AB%20%E3%81%A9%E3%82%93%E3%81%AA%20%E3%82%82%E3%81%AE%20%E3%81%8C%20%E3%81%A7%E3%81%8D%E3%81%9F%20%E3%81%8B%20%E3%81%A3%E3%81%A6%20%E3%81%84%E3%81%86%20%E3%81%AE%20%E3%82%92%20%E8%A6%8B%E3%81%9B%E3%81%9F%E3%81%84%20%20%E5%8B%95%E7%94%BB%20%E3%82%92%20%E3%81%A1%E3%82%87%E3%81%A3%E3%81%A8%20%E8%A6%8B%E3%81%A6%20%E3%81%84%E3%81%BE%E3%81%97%E3%81%9F%20%20%E9%95%B7%E3%81%84%20%20%E3%81%9D%E3%81%86%20%20%E3%81%A9%E3%81%86%20%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%20%E3%81%8B%20%E3%80%82%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%AE%20%E3%81%86%E3%81%A1%20%E3%81%95%E3%81%9B%E3%82%8B%20%E3%81%A7%E3%82%82%20%E3%81%99%E3%82%8B%20%E3%81%8B%E3%81%AA%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%8C%20%E3%81%98%E3%82%83%E3%81%A

106.129.121.5 - - [22/Oct/2017 05:07:26] "POST /exchangejk/%20%E3%82%B0%E3%83%AD%E3%83%BC%E3%83%90%E3%83%AB%E5%8C%96%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%20%E4%BC%9A%E7%A4%BE%20%E3%81%A7%20%E8%B2%B7%E3%81%88%E3%82%8B%20%E6%89%8B%E7%B6%9A%E3%81%8D%20%E3%81%A7%20%E5%AE%9F%E9%9A%9B%20%E3%81%AB%20%E3%81%A9%E3%82%93%E3%81%AA%20%E3%82%82%E3%81%AE%20%E3%81%8C%20%E3%81%A7%E3%81%8D%E3%81%9F%20%E3%81%8B%20%E3%81%A3%E3%81%A6%20%E3%81%84%E3%81%86%20%E3%81%AE%20%E3%82%92%20%E8%A6%8B%E3%81%9B%E3%81%9F%E3%81%84%20%20%E5%8B%95%E7%94%BB%20%E3%82%92%20%E3%81%A1%E3%82%87%E3%81%A3%E3%81%A8%20%E8%A6%8B%E3%81%A6%20%E3%81%84%E3%81%BE%E3%81%97%E3%81%9F%20%20%E9%95%B7%E3%81%84%20%20%E3%81%9D%E3%81%86%20%20%E3%81%A9%E3%81%86%20%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%20%E3%81%8B%20%E3%80%82%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%AE%20%E3%81%86%E3%81%A1%20%E3%81%95%E3%81%9B%E3%82%8B%20%E3%81%A7%E3%82%82%20%E3%81%99%E3%82%8B%20%E3%81%8B%E3%81%AA%20D_%E3%82%A2%E3%83%8E%E3%83%BC%20%E3%81%8C%20%E3%81%98%E3%82%83%E3%81%A